In [85]:
# testing

# tests for getMarketData

# len(my_list) == len(set(my_list))  # if true, the set has only unique values

In [86]:
import requests
import string 
import pandas as pd
import time

In [87]:
# int -> response
# returns the web response from the PI API
def getMarketRespose(market_id):
    api_url = "https://www.predictit.org/api/marketdata/markets/"
    joined_url = api_url + str(market_id)
    
    response = requests.get(joined_url)  # fetch market data
    if("null" == response.text):
        print("Bad predictItID -- No market with that ID")
    return response

In [88]:
# None -> list
# returns a list of all open markets 
def getAllOpenMarketIDs():
    # commenting out so I'm not spamming the API
    # api_url = "https://www.predictit.org/api/marketdata/all/"
    # 
    # response = requests.get(api_url)  # fetch market data
    if(response.status_code != 200):
        print("Please wait...")
        time.sleep(3)
        print("Please try again now. If this issue persists, the API access may have changed.")

    xml_response = response.json()
    all_markets_df = pd.DataFrame.from_dict(xml_response)


    market_id_list = []  
    
    for market_index in range(len(all_markets_df)):
        # print(all_markets_df["markets"][market_index]["id"])
        market_id = all_markets_df["markets"][market_index]["id"]
        market_id_list.append(market_id)

    return market_id_list

In [89]:
# TODO: change to reference the all dataframe so I'm not making a ton of requests
# 2.7 kB individual market vs 453 kB for all 
# int -> dataframe
def getMarketDataFrame(market_id):
    response = getMarketRespose(market_id)  
    xml_response = response.json()
    
    market_df = pd.DataFrame.from_dict(xml_response)
    return market_df

In [90]:
# int -> print data 
def printMarketData(market_id):
    # market_df = getMarketDataFrame(market_id)  # whole market, including 
    
    try:
        print("Market:\t", market_df["shortName"][0])  # every market has a shortname
    except KeyError:
        print("That market doesn't have a key \"shortName\"")
        return
    tabspacing = "\t\t"
    print("name", tabspacing, "Last Close Price")
    
    
    for contract_index in range(0, len(market_df)):
        all_contracts = market_df["contracts"][contract_index]
        print(all_contracts["name"], end=tabspacing)
        print(all_contracts["lastClosePrice"], end="\t")
        print()  # newline


In [91]:

# int -> print data 
def getBetterMarketDF(market_id):
    market_df = getMarketDataFrame(market_id)  # whole market, including 
    
    try:
        print("Market:\t", market_df["shortName"][0])
    except KeyError:
        print("That market doesn't have a key \"shortName\"")
        return
    tabspacing = "\t\t"
    print("name", tabspacing, "Last Close Price")
    
    
    for contract_index in range(0, len(market_df)):
        all_contracts = market_df["contracts"][contract_index]
        print(all_contracts["name"], end=tabspacing)
        print(all_contracts["lastClosePrice"], end="\t")
        print()  # newline







In [92]:
# TODO? image grabbing function
# from PIL import Image
# url = market_df["image"][0]
# im = Image.open(requests.get(url, stream=True).raw)


In [93]:
# None -> list
# returns a list of all open markets 
def getAllMarketID():
    xml_response = response.json()
    all_markets_df = pd.DataFrame.from_dict(xml_response)

    market_id_list = []  

    for market_index in range(len(all_markets_df)):
        # print(all_markets_df["markets"][market_index]["id"])
        market_id = all_markets_df["markets"][market_index]["id"]
        market_id_list.append(market_id)
        
    return market_id_list

In [94]:
# request data for entire market 
# None -> response
def getAllMarketsResponse():
    api_url = "https://www.predictit.org/api/marketdata/all/"
    response = requests.get(api_url)  # fetch market data
    if(response.status_code != 200):
        print("Please wait for API cooldown")
        
        for second in  list(range(10))[::-1]:
            time.sleep(1)
            print(second)
        print("Please try again now. If the issue persists, API access may have changed.")
        return None
    return response

In [95]:
# None -> list of all the open markets
def getAllMarketsList():
    all_markets_response = getAllMarketsResponse()
    return all_markets_response.json()["markets"]

def fetchIfNoMarketData(market_data=False):
    if market_data is False:
        print("Now fetching market data")
        market_data = getAllMarketsList()
        return market_data
    else:
        return market_data
    
    
    
def getIdIndexList( market_data=False):
    market_data = fetchIfNoMarketData(market_data)
    id_index_list = []  # 
    
    for mkt_index in range(len(market_data)):
        id_index_list.append(market_data[mkt_index]["id"])
        # print(id_index_list)
    return id_index_list

def getContractsDataFrameFromIndex( mkt_index, market_data=False):
    market_data = fetchIfNoMarketData(market_data)
    # this references the first market in the list of all markets
    market_df = pd.DataFrame.from_dict(market_data[ mkt_index ]["contracts"])
    return market_df

def getContractsDataFrame( mkt_id, market_data=False):
    market_data = fetchIfNoMarketData(market_data)
     # use the mkt_id to find the market index
    market_index_list = getIdIndexList(market_data=market_data)
    
    market_df = getContractsDataFrameFromIndex(market_index_list.index(mkt_id), market_data=market_data)
    return market_df

def concatMarketString(string):
    return "market"+string.capitalize()

# append "market" in front of each column in a dataframe if it doesn't already have the label
def labelMarket(df) -> pd.DataFrame:
    if(market_df.columns[0][0:6] == "market"):  # if marketId is already labeled, they're every column is already labeled (hopefully)
        return df
    else:
        return df.rename(concatMarketString, axis="columns")

# """
# TODO
# [ ] prepend the general market data frame with the contract specific dataframe
# [ ] look at sensible plotting options. Format data in a way that's ammenable to those options
# [ ] make csv saving job so that I don't need to worry about 
# """


In [96]:
# float, float -> bool
def inSweetSpot( value, sweet_spot=0.95, threshold=0.05 ):
    if( sweet_spot - threshold <= value <= sweet_spot+threshold):
        return True
    else:
        return False


In [97]:
# identify markets with negative risk

# whole_market_data = getAllMarketsList()
senators_vote = getContractsDataFrame(7054, market_data=whole_market_data)

filtered_buy_no = senators_vote.filter(["name", "bestBuyNoCost"])

display(filtered_buy_no)
cost = filtered_buy_no["bestBuyNoCost"].sum()
print("total cost to buy one of each no contract: ", cost)

winning_contract = 1  # 51 or 52 vote yes or something like that 

for contract in range(len(filtered_buy_no)):
    filtered_buy_no.at[contract, "bestBuyNoCost"] = 1
    
filtered_buy_no.at[winning_contract, "bestBuyNoCost"] = 0

display(filtered_buy_no)
#print(zero_scenario)

,name,bestBuyNoCost
0,50 or fewer,0.87
1,51 or 52,0.91
2,53 or 54,0.86
3,55 or 56,0.84
4,57 or 58,0.87
5,59 or 60,0.92
6,61 or 62,0.94
7,63 or 64,0.95
8,65 or 66,0.95
9,67 or more,0.79


total cost to buy one of each no contract:  8.899999999999999


,name,bestBuyNoCost
0,50 or fewer,1.0
1,51 or 52,0.0
2,53 or 54,1.0
3,55 or 56,1.0
4,57 or 58,1.0
5,59 or 60,1.0
6,61 or 62,1.0
7,63 or 64,1.0
8,65 or 66,1.0
9,67 or more,1.0
